In [ ]:
!pip install langdetect
!pip install nltk

In [26]:
import pandas as pd
import requests
import json
import os
from bs4 import BeautifulSoup
import re
from queue import Queue
from threading import Thread
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

#from nltk import wordpunct_tokenize
from langdetect import detect
from nltk import word_tokenize
from nltk.corpus import stopwords
import re
from difflib import SequenceMatcher

In [2]:
json1_file = open(os.getcwd() + '/Credentials2.json')
json1_str = json1_file.read()
credential = json.loads(json1_str)
Genius_Token = credential['Genius_API']['Access_Token']

In [3]:
def request_song_info(song_id,Genius_Token):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + Genius_Token}
    song_url = base_url + '/songs/' + str(song_id)
    response = requests.get(song_url, headers = headers)

    return response

In [4]:
def get_song_id(ix,song_title,artist_title,Genius_Token):
    
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + Genius_Token}
    search_url = base_url + '/search'
    data = {'q': str(song_title) + ' ' + str(artist_title)}
    response = requests.get(search_url, data = data, headers=headers)

    if response.ok == True:
        song_info = response.json()
        if len(song_info['response']['hits']) != 0:
            song_id = song_info['response']['hits'][0]['result']['id']

        else:
            song_id = 'Not Found'
        return (ix,song_id)
    else:
        return "Response Error!"
    

In [5]:
def is_legal_string(test_string):
    legal_characters = r'\w+$'
    r = re.compile(legal_characters)
    
    if re.match(r, test_string):
        return 1
    else:
        return 0

In [6]:
def scrap_song_url(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()

    return lyrics

In [7]:
def get_lyrics(song_id):
    response = request_song_info(song_id,Genius_Token)
    if response.ok == True:
        info = response.json()
        if len(info['response']['song']['url']) != 0:
            song_url = info['response']['song']['url']
            lyrics = scrap_song_url(song_url)
        else:
            lyrics = 'Not Found'
    else:
        return "Response Error!"
    
    
    return lyrics

In [8]:
def get_release_date(song_id):
    song_info = request_song_info(song_id,Genius_Token)
    song_info = song_info.json()
    
    return song_info['response']['song']['release_date']

In [9]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [10]:
tracks = pd.read_csv(r'Clean Data/track_with_artists.csv')

    First get song_id from Genius

In [114]:
result = list()
def worker():
    while True:
        item = q.get()
        result.append(get_song_id(item[0],item[1],item[2],Genius_Token))
        q.task_done()

q = Queue()
for i in range(8):
    t = Thread(target=worker)
    t.daemon = True
    t.start()

for item in tracks.iterrows():
    q.put((item[0],item[1]['TITLE'],item[1]['ARTIST']))

q.join() 

In [139]:
result = pd.DataFrame({'ix':[i[0] for i in result],'song_id':[i[1] for i in result]})
tracks_with_song_ids = pd.merge(result,tracks,left_on = 'ix',right_index = True, how = 'inner').drop('ix',axis = 1)
tracks_with_song_ids.to_csv(r'tracks_with_song_ids.csv',index = False)

In [ ]:
#song song_ids are duplicated, drop duplicates
song_id_no_duplicates = tracks_with_song_ids[tracks_with_song_ids['song_id'].duplicated() == False]
song_id_no_duplicates.to_csv('Clean Data/song_id_no_duplicates.csv',index = False)

In [168]:
print("unique song_ids found:{}".format(len(song_id_no_duplicates)))

unique song_ids found:286110


In [170]:
song_id_no_duplicates.head()

,song_id,ISRC,SPOTIFY_ID,TITLE,ARTIST
0,3035330,USRC11702155,1j4kHkkpqZRBwE0A4CN4Yv,Dusk Till Dawn - Radio Edit,ZAYN
1,3646568,USUM71805170,7dt6x5M1jzdTEt8oCbisTK,Better Now,Post Malone
2,2942139,USUM71710087,7wGoVu4Dady5GV0Sv4UIsx,rockstar,Post Malone
3,2992529,USSM11706905,0ofbQMrRDsUaVKq2mGLEAb,Havana,Camila Cabello
4,3188009,USRC11701901,7vGuf3Y35N4wmASOKLUVVU,Silence,Marshmello


    Validation song_id and song title
<p> Use Genius API again to request the song title from song_id and compare with the original title. If the similarity between two titles are less then 0.8, see that as same song titles</p>

In [ ]:
verify_list = []
for item in song_id_no_duplicates.iterrows():
    response = request_song_info(int(item[1]['song_id']),Genius_Token)
    if response.ok == True:
        verify_list.append(response.json()['response']['song']['title'])
    else:
        verify_list.append("Not Found")

In [ ]:
if len(verify_list) == len(song_id_no_duplicates):
    song_id_no_duplicates['verified_title'] = verify_list

In [ ]:
song_id_no_duplicates['if_verified'] = song_id_no_duplicates.apply(lambda x: 1 if similar(x['TITLE'],x['verified_title']) >= 0.8 else 0 , axis = 1)

In [ ]:
verified_df = song_id_no_duplicates[song_id_no_duplicates['if_verified'] == 1]
verified_df.to_csv('verified_song_ids.csv',index=False)

    Scrap lyrics by song_ids

In [ ]:
result = list()
def worker():
    while True:
        item = q.get()
        result.append([item[0],get_lyrics(item[0])])
        q.task_done()

q = Queue()
for i in range(8):
    t = Thread(target=worker)
    t.daemon = True
    t.start()

for item in verified_df.iterrows():
    q.put(item[1]['song_id'])

q.join() 

In [ ]:
result_lyrics = pd.DataFrame({'song_id':[i[0] for i in result],'lyrics':[i[1] for i in result]})
lyrics_full = result_lyrics.merge(verified_df,on='song_id',how='inner')
#some lyrics are empty values, drop them
lyrics_full = lyrics_full[lyrics_full['lyrics'].isnull() == False]
lyrics_full.to_csv(r'lyrics_full.csv',index=False)

    There are song non-english lyrics, so we have to filter out only english songs and english lyrics

In [ ]:
s = []
for item in lyrics_full.iterrows():
    cleanString = re.sub('[^A-Za-z0-9]+',' ',item[1]['lyrics'])
    if len(cleanString) > 200:   
        if detect(cleanString)=='en':
            s.append(item[1]['song_id'])

In [ ]:
ddf=pd.DataFrame()
ddf['song_id']=s
english_lyrics =ddf.merge(lyrics_full,on='song_id',how='inner')
english_lyrics.to_csv(r'verified_english_lyrics.csv',index=False)

    Merge with labels

In [23]:
df = pd.read_csv(r'verified_english_lyrics.csv')

In [25]:
label = pd.read_csv(r'Raw Data/max popularity by track.csv')

/Users/Stella/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [47]:
temp = label.drop_duplicates('track_external_id')

In [54]:
df_with_label = df.merge(temp,left_on='ISRC',right_on='track_external_id',how='inner').drop(['track_id','track_external_id'],axis = 1)

In [11]:
#df_with_label.to_csv(r'verified_english_lyrics_with_label.csv')
df = pd.read_csv(r'verified_english_lyrics_with_label.csv')

In [12]:
df.head()

,Unnamed: 0,song_id,ISRC,SPOTIFY_ID,TITLE,ARTIST,lyrics,max_popularity
0,0,1065560,CAU117502590,36D5CiNfl2k043KtVHyEOr,I Wouldn't Want to Lose Your Love,April Wine,"\n\nEasy now, don't let our love go wrong\nI'd...",21
1,1,815171,AUSM09600084,4W0nfmA4wtvaRbLaLB4Kx0,Greatest Gift,Tina Arena,\n\nI awake to my beating heart\nYou save my s...,23
2,2,877836,GBSWE1448094,291NQX66yyvKbWZriNtpVH,Designing Heaven,Heaven 17,\n\nHow demanding life can be\nWith a taste fo...,11
3,3,1489944,GBAAW9700265,5JAjbnW67TWgPRDERCL1BD,Speed of Light,Teenage Fanclub,\n\n[Verse 1]\nDrive an easy road\nIf you're l...,24
4,4,928717,ushm80469236,3sRRy8ScmH2dMjPy9WiVum,Queen of Pain,Devil Doll,"\n\nI'm the queen of pain, I can make it thund...",26


In [54]:
def get_producer(song_id):
    response = request_song_info(song_id,Genius_Token)
    if response.ok == True:
        info = response.json()
        song_info = info['response']['song']
        if 'producer_artists' in song_info.keys():
            producer_list = song_info['producer_artists']
            result = [prod['name'] for prod in producer_list]
        else:
            result = 'missing producer info'
    else:
        result = 'Connection Error'
    return result
                    

In [27]:
result = list()

for item in df.iterrows():
    result.append(get_producer(item[1]['song_id']))

In [16]:
if len(result) == len(df):
    df['producer'] = result
else:
    print("lengths don't match, please double check!")

/Users/Stella/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [40]:
df.head()

,song_id,ISRC,SPOTIFY_ID,TITLE,ARTIST,lyrics,max_popularity,producer
0,1065560,CAU117502590,36D5CiNfl2k043KtVHyEOr,I Wouldn't Want to Lose Your Love,April Wine,"\n\nEasy now, don't let our love go wrong\nI'd...",21,"[Dino Danelli, Gene Cornish]"
1,815171,AUSM09600084,4W0nfmA4wtvaRbLaLB4Kx0,Greatest Gift,Tina Arena,\n\nI awake to my beating heart\nYou save my s...,23,[]
2,877836,GBSWE1448094,291NQX66yyvKbWZriNtpVH,Designing Heaven,Heaven 17,\n\nHow demanding life can be\nWith a taste fo...,11,[]
3,1489944,GBAAW9700265,5JAjbnW67TWgPRDERCL1BD,Speed of Light,Teenage Fanclub,\n\n[Verse 1]\nDrive an easy road\nIf you're l...,24,"[Teenage Fanclub, David Bianco]"
4,928717,ushm80469236,3sRRy8ScmH2dMjPy9WiVum,Queen of Pain,Devil Doll,"\n\nI'm the queen of pain, I can make it thund...",26,[]


In [41]:
df.to_csv(r'verified_english_lyrics_with_producers.csv',index=False)

    Get SongWriters

In [57]:
def get_writer(song_id):
    response = request_song_info(song_id,Genius_Token)
    if response.ok == True:
        info = response.json()
        song_info = info['response']['song']
        if 'writer_artists' in song_info.keys():
            writer_list = song_info['writer_artists']
            result = [w['name'] for w in writer_list]
        else:
            result = 'missing writer info'
    else:
        result = 'Connection Error'
    return result
                    

In [10]:
df = pd.read_csv(r'final_matrix.csv')

In [11]:
df.head()

,ARTIST,ISRC,SPOTIFY_ID,TITLE,lyrics,song_id,producer,number_of_tracks,number_of_albums,number_of_playlists,...,Metal,Other,Pop,R&B,Reggae,Rock/Alt,Rock/Alt/Pop,Singer-Songwriter,genre_missing,Explicit
0,Yo La Tengo,USMTD0985602,158mw0CsuSHxTVWpMwN2Lp,Avalon or Someone Very Similar,"\n\nAvalon, how did these things happen? Avalo...",1369717,['Roger Moutenot'],143,40,649,...,0,1,1,0,0,1,0,0,0,0
1,Yo La Tengo,USGKX9802104,7iX2vtYJlemPod7080OxTG,Swing For Life,"\n\nOh, true rider, fasten me\nI saw a mystery...",1063052,[],143,40,649,...,0,1,1,0,0,1,0,0,0,0
2,Yo La Tengo,USMTD1810576,2YmjVaikl30rmZZzlGTwPE,Ashes,"\n\nI put spots on a bird of paradise, snake e...",3596137,['Yo La Tengo'],143,40,649,...,0,1,1,0,0,1,0,0,0,0
3,Smrtdeath,QZANL1773974,4sBj3qUIPBkIMcRVlh6iXI,Let It Bleed,"\n\n[Intro]\nSethany, Sethany, Sethany\nYeah, ...",3150542,['Taxpurposes'],10,10,30,...,0,0,0,0,0,0,0,0,0,1
4,Smrtdeath,QZANL1773973,3gjrflTTriqZ9d30uSF4hP,Went & Got Some More,"\n\n[Hook: Smrtdeath]\nI just lost some drugs,...",3256462,"['Eternal Night', 'Sorry If I Waste Your Time']",10,10,30,...,0,0,0,0,0,0,0,0,0,1


In [18]:
result = list()

for item in df.iterrows():
    result.append(get_writer(item[1]['song_id']))

In [19]:
if len(df) == len(result):
    df['writer'] = result
else:
    print('lengths not match, please double check')

In [22]:
df.to_csv(r'final_matrix_with_writers.csv',index=False)

    Scrape lyrics for popular tracks(a-artists' tracks)

In [10]:
folder_path = '/'.join(os.getcwd().split('/')[:-1]) + '/Clean Data/'
a_tracks = pd.read_csv(folder_path + 'a-artitss_tracks.csv')

In [13]:
a_tracks.head()

,ISRC,SPOTIFY_ID,TITLE,ARTIST,TEMPO,ENERGY,DANCE,LOUD,VALENCE,ACOUSTIC,SPEECHINESS,INSTRUMENTALNESS,KEY,LIVENESS,TIME_SIGNATURE,MODE,POP,Seconds,Lean-Back
0,USCM51800207,0h1W19pS59KtEd7aDzF58i,In My Feelings,Drake,91.022,0.619,0.833,-5.822,0.366,0.0496,0.0971,0.000087,8,0.300,4,1,100,217925,0
1,USCM51800078,7liAO2OqrFMVvM7oiLo0m0,Nice For What,Drake,186.885,0.899,0.603,-6.558,0.813,0.1050,0.1410,0.000431,8,0.106,4,1,100,210925,0
2,USCM51800005,61T2H6w0ggQdhCvVegLl99,God's Plan,Drake,77.175,0.451,0.758,-9.454,0.353,0.0273,0.1040,0.000095,7,0.566,4,1,100,198960,0
3,USCM51800004,6T8cJz5lAqGer9GUHGyelE,God's Plan,Drake,77.179,0.448,0.755,-9.587,0.318,0.0283,0.1070,0.000033,7,0.514,4,1,100,198960,0
4,USCM51800077,1cTZMwcBJT0Ka3UJPXOeeN,Nice For What,Drake,93.350,0.913,0.567,-6.471,0.792,0.0934,0.0736,0.000124,8,0.114,4,1,100,210925,0


In [14]:
result = list()

for item in a_tracks.iterrows():
    result.append(get_song_id(item[0],item[1]['TITLE'],item[1]['ARTIST'],Genius_Token))

In [17]:
if len(result) == len(a_tracks):
    a_tracks['song_id'] = [item[1] for item in result]

In [19]:
a_tracks = a_tracks[a_tracks['song_id'] != 'Not Found']

In [20]:
verify_list = []

for item in a_tracks.iterrows():
    response = request_song_info(int(item[1]['song_id']),Genius_Token)
    if response.ok == True:
        verify_list.append(response.json()['response']['song']['title'])
    else:
        verify_list.append("Not Found")

In [21]:
if len(verify_list) == len(a_tracks):
    a_tracks['verify_title'] = verify_list

/Users/Stella/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
a_tracks['if_verified'] =  a_tracks.apply(lambda x: 1 if similar(x['TITLE'],x['verify_title']) >= 0.8 else 0 , axis = 1)

/Users/Stella/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
a_tracks['if_verified'].value_counts()

1    200
0     69
Name: if_verified, dtype: int64

In [30]:
a_tracks[a_tracks['if_verified'] == 0][['TITLE','verify_title']]

,TITLE,verify_title
7,River (feat. Ed Sheeran),Eminem - River (feat. Ed Sheeran) [Deutsche Üb...
8,River (feat. Ed Sheeran),Eminem - River (feat. Ed Sheeran) [Deutsche Üb...
16,Friends (with BloodPopÂ®),OVO Sound Radio Episode 54 Tracklist
18,Mi Gente (feat. BeyoncÃ©),Mi Gente (Beyoncé Remix)
19,Bed (feat. Ariana Grande),New Music Friday 6/15/2018
20,All The Stars (with SZA),Kendrick Lamar & SZA - All The Stars (Tradução)
21,Bed (feat. Ariana Grande),New Music Friday 6/15/2018
22,All The Stars (with SZA),Kendrick Lamar & SZA - All The Stars (Tradução)
42,Watch (feat. Lil Uzi Vert & Kanye West),New Music Friday 5/4/18
43,Watch (feat. Lil Uzi Vert & Kanye West),New Music Friday 5/4/18


In [36]:
idx_list = [16,19,21,42,43,82,83,92,93,99,100,101,102,103,108,110,112,113,117,132,155,138,151,162,164,179,185,188,190,194,197,215,216,226,236,238,248,249,264,269,276,284,285,293,294,295,298]

In [37]:
a_tracks = a_tracks[~a_tracks.index.isin(idx_list)]

In [41]:
a_tracks = a_tracks[['ISRC', 'SPOTIFY_ID', 'TITLE', 'ARTIST','song_id','POP']]

In [42]:
a_tracks.head()

,ISRC,SPOTIFY_ID,TITLE,ARTIST,song_id,POP
0,USCM51800207,0h1W19pS59KtEd7aDzF58i,In My Feelings,Drake,3807759,100
1,USCM51800078,7liAO2OqrFMVvM7oiLo0m0,Nice For What,Drake,3580132,100
2,USCM51800005,61T2H6w0ggQdhCvVegLl99,God's Plan,Drake,3315890,100
3,USCM51800004,6T8cJz5lAqGer9GUHGyelE,God's Plan,Drake,3315890,100
4,USCM51800077,1cTZMwcBJT0Ka3UJPXOeeN,Nice For What,Drake,3580132,100


In [45]:
lyrics_list = []

for item in a_tracks.iterrows():
    lyrics_list.append(get_lyrics(item[1]['song_id']))

In [46]:
if len(lyrics_list) == len(a_tracks):
    a_tracks['lyrics'] = lyrics_list

In [50]:
a_tracks = a_tracks[a_tracks['song_id'].duplicated() == False]

In [52]:
a_tracks.head()

,ISRC,SPOTIFY_ID,TITLE,ARTIST,song_id,POP,lyrics
0,USCM51800207,0h1W19pS59KtEd7aDzF58i,In My Feelings,Drake,3807759,100,"\n\n[Intro: Drake]\nTrap, TrapMoneyBenny\nThis..."
1,USCM51800078,7liAO2OqrFMVvM7oiLo0m0,Nice For What,Drake,3580132,100,\n\n[Intro: Big Freedia]\nI wanna know who mot...
2,USCM51800005,61T2H6w0ggQdhCvVegLl99,God's Plan,Drake,3315890,100,"\n\n[Intro]\nYeah, they wishin' and wishin' an..."
7,USUM71713011,6ZMbykUu07aL2Drx988zAq,River (feat. Ed Sheeran),Eminem,3384370,98,\n\n[Refrain: Ed Sheeran]\nIch bin ein Lügner ...
9,USUM71805289,5SxkdsY1ufZzoq9iXceLw9,no tears left to cry,Ariana Grande,3649172,98,"\n\n[Intro]\nRight now, I'm in a state of mind..."


In [55]:
producer_list = []

for item in a_tracks.iterrows():
    producer_list.append(get_producer(item[1]['song_id']))

In [56]:
if len(producer_list) == len(a_tracks):
    a_tracks['producer'] = producer_list

In [58]:
writer_list = []

for item in a_tracks.iterrows():
    writer_list.append(get_writer(item[1]['song_id']))

In [59]:
if len(writer_list) == len(a_tracks):
    a_tracks['writer'] = writer_list

In [64]:
a_tracks.to_csv(folder_path + '/a_tracks.csv',index=False)